# Setup

In [1]:
import os
import openai
import requests
import os

openai.api_key = ""
client = openai.OpenAI(api_key= os.environ.get("OPENAI_API_KEY"))


def llm(history, stop=["\n"]):
    response = client.chat.completions.create(
                model='gpt-4o',
                messages=history,
                # temperature=0.5,
                # top_p=0.9,
                # n=1,
                stop=stop,
            )


    return response.choices[0].message.content.strip()

In [2]:
from hotpotqa import wrappers, wikienv
env = wikienv.WikiEnv()
env = wrappers.FeverWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

# ReAct

In [11]:
import json
import sys

folder = './prompts/'
prompt_file = 'fever.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

webthink_prompt = prompt_dict['webact_simple3']

def webthink(idx=None, prompt=webthink_prompt, to_print=True):
    question = env.reset(idx=idx)
    history = []
    history.append({"role": "user", "content": webthink_prompt})

    if to_print:
        print(idx, question)


    n_calls, n_badcalls = 0, 0



    for i in range(1, 11):
        prompt = f"Action {i}:"
        if i == 1:
            prompt = question + "\n" + prompt

        if i == 10:
            prompt = 'This is the final round, you must provide a final answer using Finish[...] action!\n' + prompt

        history.append({"role": "user", "content": prompt})
        n_calls += 1

        action_str = llm(history, stop=[f"\nObservation {i}:"])
        try:
            #Extract Action
            action =  action_str.split(f"Action {i}:")[-1].strip()
        except:
            print('ohh...', action_str)
            n_badcalls += 1
            n_calls += 1
            history.append({"role": "user", "content": f"Action {i}:"})
            action = llm(history, stop=[f"\n"]).strip()

        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Action {i}: {action}\nObservation {i}: {obs}\n"
        history.append({"role": "assistant", "content": step_str})
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info


import random
import time
idxs = list(range(7405))
random.Random(233).shuffle(idxs)

rs = []
infos = []
old_time = time.time()

import os
baseline_check_point_file = 'baseline_checkpoints111.json'
if os.path.exists(baseline_check_point_file):
    with open(baseline_check_point_file, 'r') as f:
        baseline_checkpoints = json.load(f)
        rs = baseline_checkpoints['rs']
        infos = baseline_checkpoints['infos']
        start_idx = len(baseline_checkpoints['rs'])

else:
    baseline_checkpoints = {
        'rs': [],
        'infos': []
    }
    start_idx = 0

checkpoint_interval = 1
for i in idxs[0:200]:
    try:
        r, info = webthink(i, to_print=True)
        rs.append(info['em'])

    except Exception as e:
        print(f"Error at index {i}: {e}")
        r = 0
        info = {}
        rs.append(0)

    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()
    if i % checkpoint_interval == 0:
        baseline_checkpoints['rs'] = rs
        baseline_checkpoints['infos'] = infos
        with open(baseline_check_point_file, 'w') as f:
            json.dump(baseline_checkpoints, f)
        print('Checkpoint saved.')

3687 Claim: Paramore is not from Tennessee.
Action 1: Search[Paramore]
Observation 1: Paramore is an American rock band formed in Franklin, Tennessee, in 2004. Since 2017, the band's lineup includes lead vocalist Hayley Williams, lead guitarist Taylor York, and drummer Zac Farro. Williams and Farro are founding members of the group, while York, a high school friend of the original lineup, joined in 2007. The band has had multiple lineup changes, with Williams being the only constant member.. The band was signed to Fueled by Ramen, a subsidiary of Atlantic Records (which is owned by Warner Music Group.) Williams was signed to Atlantic separately, as she was scouted when she was a teenager.

Action 2: Finish[REFUTES]
Observation 2: Episode finished, reward = 1


{'steps': 2, 'answer': 'REFUTES', 'gt_answer': 'REFUTES', 'question_idx': 3687, 'em': 1, 'reward': 1, 'f1': 1} 

1 1 1.0 14.261192560195923
-----------

Checkpoint saved.
6238 Claim: Eric Church is a sound mixer.
Action 1: Search